# Upper Limits with Python

This sample analysis shows a way to determine an upper limit on the GeV emission from Swift J164449.3+573451 similar to what was done in [Burrows et al. (Nature 476, page 421)](http://www.nature.com/nature/journal/v476/n7361/full/nature10374.html).

To compute the upper limit, we use the profile likelihood method. This entails scanning in values of the normalization parameter, fitting with respect to the other remaining free parameters, and plotting the change in log-likelihood as a function of flux.

Assuming 2$\cdot$log-likelihood behaves asymptotically as chi-square, a 90% confidence region will correspond to a change in log-likelihood of 2.71/2.

Note that this change in log-likelihood corresponds to a two-sided confidence interval. Since we are interested in an upper-limit, this change in log-likelihood actually corresponds to a 95% CL upper-limit. See [Rolke et al. (2005)](https://arxiv.org/abs/physics/0403059) for more details.

We will first cover an unbinned example and at the end of the page we include modifications for binned data. This tutorial assumes that you have gone through the standard [likelihood analysis](https://github.com/fermi-lat/AnalysisThreads/blob/master/SourceAnalysis/3.PythonLikelihood/python_tutorial.ipynb).

### Get the Data

For this thread the original data were extracted from the [LAT data server](http://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/LATDataQuery.cgi) with the following selections (these selections are similar to those in the paper):

```
Search Center (RA,Dec) = (251.2054,57.5808)
Radius = 30 degrees
Start Time (MET) = 322963202 seconds (2011-03-28T00:00:00)
Stop Time (MET) = 323568002 seconds (2011-04-04T00:00:00)
Minimum Energy = 100 MeV
Maximum Energy = 300000 MeV
```

You will need the following files:
```
L181102105258F4F0ED2772_PH00.fits
L181102105258F4F0ED2772_SC00.fits
gll_iem_v07.fits
iso_P8R3_SOURCE_V3.txt
```

Run the code cell below to download them.

In [13]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/pyLikelihood/L181102105258F4F0ED2772_PH00.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/pyLikelihood/L181102105258F4F0ED2772_SC00.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/software/aux/gll_iem_v07.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/software/aux/iso_P8R3_SOURCE_V3_v1.txt

--2025-09-15 10:12:22--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/pyLikelihood/L181102105258F4F0ED2772_PH00.fits
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12029760 (11M) [application/fits]
Saving to: ‘L181102105258F4F0ED2772_PH00.fits’

L181102105258F4F0ED 100%[===================>]  11.47M   827KB/s    in 16s     

2025-09-15 10:12:42 (744 KB/s) - ‘L181102105258F4F0ED2772_PH00.fits’ saved [12029760/12029760]

--2025-09-15 10:12:42--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/pyLikelihood/L181102105258F4F0ED2772_SC00.fits
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2629440 (2.5M) [application/fits]
Saving to: ‘L

In [ ]:
!mkdir data
!mv *.fits *.txt ./data

# Perform Event Selections

You could follow the unbinned likelihood tutorial to perform your event selections using **gtlike*, **gtmktime**, etc. directly from the command line, and then use pylikelihood later.

But we're going to go ahead and use python. The `gt_apps` module provides methods to call these tools from within python. This'll get us used to using python.

So, let's jump into python:

In [15]:
import gt_apps as my_apps

We first run **gtselect** (`filter` in python):

In [16]:
my_apps.filter['evclass'] = 128
my_apps.filter['evtype'] = 3
my_apps.filter['ra'] = 251.2054
my_apps.filter['dec'] = 57.5808
my_apps.filter['rad'] = 10
my_apps.filter['emin'] = 100
my_apps.filter['emax'] = 300000
my_apps.filter['zmax'] = 90
my_apps.filter['tmin'] = 322963202
my_apps.filter['tmax'] = 323568002
my_apps.filter['infile'] = './data/L181102105258F4F0ED2772_PH00.fits'
my_apps.filter['outfile'] = './data/SwiftJ1644_filtered.fits'

In [17]:
my_apps.filter.run()

time -p gtselect infile=./data/L181102105258F4F0ED2772_PH00.fits outfile=./data/SwiftJ1644_filtered.fits ra=251.2054 dec=57.5808 rad=10.0 tmin=322963202.0 tmax=323568002.0 emin=100.0 emax=300000.0 zmin=0.0 zmax=90.0 evclass=128 evtype=3 convtype=-1 phasemin=0.0 phasemax=1.0 evtable="EVENTS" chatter=2 clobber=yes debug=no gui=no mode="ql"
Done.
real 0.47
user 0.31
sys 0.03


Now, we need to find the GTIs. This is accessed within python via the `maketime` object:

In [18]:
my_apps.maketime['scfile'] = './data/L181102105258F4F0ED2772_SC00.fits'
my_apps.maketime['filter'] = '(DATA_QUAL>0)&&(LAT_CONFIG==1)'
my_apps.maketime['roicut'] = 'no'
my_apps.maketime['evfile'] = './data/SwiftJ1644_filtered.fits'
my_apps.maketime['outfile'] = './data/SwiftJ1644_filtered_gti.fits'

In [19]:
my_apps.maketime.run()

time -p gtmktime scfile=./data/L181102105258F4F0ED2772_SC00.fits sctable="SC_DATA" filter="(DATA_QUAL>0)&&(LAT_CONFIG==1)" roicut=no evfile=./data/SwiftJ1644_filtered.fits evtable="EVENTS" outfile="./data/SwiftJ1644_filtered_gti.fits" apply_filter=yes overwrite=no header_obstimes=yes tstart=0.0 tstop=0.0 gtifile="default" chatter=2 clobber=yes debug=no gui=no mode="ql"
real 0.16
user 0.08
sys 0.02


# Livetime Cube and Exposure Map

Let's compute the livetime cube and exposure map.

### Livetime Cube

In [20]:
my_apps.expCube['evfile'] = './data/SwiftJ1644_filtered_gti.fits'
my_apps.expCube['scfile'] = './data/L181102105258F4F0ED2772_SC00.fits'
my_apps.expCube['outfile'] = './data/SwiftJ1644_ltCube.fits'
my_apps.expCube['zmax'] = 90
my_apps.expCube['dcostheta'] = 0.025
my_apps.expCube['binsz'] = 1

In [21]:
my_apps.expCube.run()

time -p gtltcube evfile="./data/SwiftJ1644_filtered_gti.fits" evtable="EVENTS" scfile=./data/L181102105258F4F0ED2772_SC00.fits sctable="SC_DATA" outfile=./data/SwiftJ1644_ltCube.fits dcostheta=0.025 binsz=1.0 phibins=0 tmin=0.0 tmax=0.0 file_version="1" zmin=0.0 zmax=90.0 chatter=2 clobber=yes debug=no gui=no mode="ql"
Working on file ./data/L181102105258F4F0ED2772_SC00.fits
.....................!
real 10.54
user 10.34
sys 0.12


### Exposure Map

In [22]:
from GtApp import GtApp

expCubeSun = GtApp('gtltcubesun','Likelihood')
expCubeSun.command()
my_apps.expMap['evfile'] = './data/SwiftJ1644_filtered_gti.fits'
my_apps.expMap['scfile'] ='./data/L181102105258F4F0ED2772_SC00.fits'
my_apps.expMap['expcube'] ='./data/SwiftJ1644_ltCube.fits'
my_apps.expMap['outfile'] ='./data/SwiftJ1644_expMap.fits'
my_apps.expMap['irfs'] = 'CALDB'
my_apps.expMap['srcrad'] = 20
my_apps.expMap['nlong'] = 120
my_apps.expMap['nlat'] = 120
my_apps.expMap['nenergies'] = 37

In [23]:
my_apps.expMap.run()

time -p gtexpmap evfile=./data/SwiftJ1644_filtered_gti.fits evtable="EVENTS" scfile=./data/L181102105258F4F0ED2772_SC00.fits sctable="SC_DATA" expcube=./data/SwiftJ1644_ltCube.fits outfile=./data/SwiftJ1644_expMap.fits irfs="CALDB" evtype="INDEF" srcrad=20.0 nlong=120 nlat=120 nenergies=37 submap=no nlongmin=0 nlongmax=0 nlatmin=0 nlatmax=0 chatter=2 clobber=yes debug=no gui=no mode="ql"
The exposure maps generated by this tool are meant
to be used for *unbinned* likelihood analysis only.
Do not use them for binned analyses.
real 313.60
user 303.70
sys 8.40


# Generate XML Model File

We need to create an XML file with all of the sources of interest within the Region of Interest (ROI) of SwiftJ1644 so we can correctly model the background.

We'll use the user contributed tool `LATSourceModel` package to create a model file based on the 14-year LAT catalog. You'll need to download the XML or FITS version of this file at http://fermi.gsfc.nasa.gov/ssc/data/access/lat/14yr_catalog/ and put it in your working directory. Install the [LATSourceModel](https://github.com/physicsranger/make4FGLxml) package from the [user-contributed software page](https://fermi.gsfc.nasa.gov/ssc/data/analysis/user/) by following the instructions on the linked GitHub page.

Also make sure you have the most recent galactic diffuse and isotropic model files, which can be found [here](http://fermi.gsfc.nasa.gov/ssc/data/access/lat/BackgroundModels.html).

The catalog and background models are also packaged with your installation of the ScienceTools, which can be found at: `$FERMI_DIR/refdata/fermi/galdiffuse/`.

In [24]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/access/lat/14yr_catalog/gll_psc_v32.xml

--2025-09-15 10:28:07--  https://fermi.gsfc.nasa.gov/ssc/data/access/lat/14yr_catalog/gll_psc_v32.xml
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12719321 (12M) [application/xml]
Saving to: ‘gll_psc_v32.xml’

gll_psc_v32.xml     100%[===================>]  12.13M  1.15MB/s    in 8.9s    

2025-09-15 10:28:18 (1.36 MB/s) - ‘gll_psc_v32.xml’ saved [12719321/12719321]



In [25]:
!mv *.xml ./data/

Now that we have all of the files we need, we can generate your model file in python:

In [26]:
from LATSourceModel import SourceList
mymodel = SourceList(catalog_file='./data/gll_psc_v32.xml',
                     ROI='./data/SwiftJ1644_filtered_gti.fits',
                     output_name='./data/SwiftJ1644_model.xml',
                     DR=4)
mymodel.make_model(free_radius=3,max_free_radius=5)

Creating spatial and spectral model from the 4FGL DR-4 catalog: data/gll_psc_v32.xml.
Added 221 point sources and 0 extended sources.
Building ds9-style region file...done!
File saved as data/ROI_SwiftJ1644_model.reg.


In the step above, some additional options provided by the make4FGLxml.py tool have been invoked to allow the likelihood tool to obtain an initial fit. The tool can be read with a text editor (vim, emacs, etc.) to find explantions for these and other parameters near the top of the file.

# Compute the diffuse source responses

The [gtdiffrsp](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtdiffrsp.txt) tool will add one column to the event data file for each diffuse source.

The diffuse response depends on the instrument response function (IRF), which must be in agreement with the selection of events, i.e. the event class and event type we are using in our analysis.

Since we are using SOURCE class, `CALDB` should use the `P8R3_SOURCE_V3` IRF for this tool.

In [27]:
import gt_apps as my_apps

my_apps.diffResps['evfile'] = './data/SwiftJ1644_filtered_gti.fits'
my_apps.diffResps['scfile'] = './data/L181102105258F4F0ED2772_SC00.fits'
my_apps.diffResps['srcmdl'] = './data/SwiftJ1644_model.xml'
my_apps.diffResps['irfs'] = 'CALDB'

In [28]:
my_apps.diffResps.run()

time -p gtdiffrsp evfile=./data/SwiftJ1644_filtered_gti.fits evtable="EVENTS" scfile=./data/L181102105258F4F0ED2772_SC00.fits sctable="SC_DATA" srcmdl=./data/SwiftJ1644_model.xml irfs="CALDB" evclsmin=0 evclass="INDEF" evtype="INDEF" convert=no chatter=2 clobber=no debug=no gui=no mode="ql"
adding source gll_iem_v07
adding source iso_P8R3_SOURCE_V3_v1
Working on...
./data/SwiftJ1644_filtered_gti.fits.....................!
real 29.80
user 29.00
sys 0.56


# Run the Likelihood Analysis

First, import the pyLikelihood module and then the UnbinnedAnalysis functions. For more details on the pyLikelihood module, check out the [pyLikelihood Usage Notes](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/python_usage_notes.html).

In [29]:
import pyLikelihood
from UnbinnedAnalysis import *

obs = UnbinnedObs('./data/SwiftJ1644_filtered_gti.fits','./data/L181102105258F4F0ED2772_SC00.fits',expMap='./data/SwiftJ1644_expMap.fits',expCube='./data/SwiftJ1644_ltCube.fits',irfs='CALDB')
like = UnbinnedAnalysis(obs,'./data/SwiftJ1644_model.xml',optimizer='Minuit')
like.tol = 0.1
likeobj = pyLike.Minuit(like.logLike)
like.fit(verbosity=0,covar=True,optObject=likeobj)
likeobj.getQuality()
like.logLike.writeXml('./data/Swift_fit1.xml')

This output corresponds to the `MINUIT` fit quality. A "good" fit corresponds to a value of `fit quality = 3`; if you get a lower value it is likely that there is a problem with the error matrix. Now we try with NewMinuit:

In [30]:
like2 = UnbinnedAnalysis(obs,'./data/Swift_fit1.xml',optimizer='NewMinuit')
like2.tol = 0.0001
like2obj = pyLike.NewMinuit(like2.logLike)
like2.fit(verbosity=0,covar=True,optObject=like2obj)

14092.597409004013

In [31]:
print(like2obj.getRetCode())

156


If you get anything other than `0` here, then NewMinuit didn't converge.

We can start by deleting sources with low or negative TS, which tend to hinder convergence. First, we delete sources with TS levels below 10 and run the fit again.

In [32]:
sourceDetails = {}

for source in like2.sourceNames():
    sourceDetails[source] = like.Ts(source)

for source,TS in sourceDetails.items():
    print(source,TS)
    if (TS < 10):
        print("Deleting...")
        like2.deleteSource(source)

like2.fit(verbosity=0,covar=True,optObject=like2obj)

print(like2obj.getRetCode())

like2.logLike.writeXml('./data/Swift_ts10.xml')

4FGL J1409.7+5940 0.00011189885117346421
Deleting...
4FGL J1410.3+6058 8.622191671747714e-07
Deleting...
4FGL J1410.5+6215 -4.306376285967417e-05
Deleting...
4FGL J1417.3+6059 -1.5848756447667256e-05
Deleting...
4FGL J1422.6+5801 2.81309949059505e-05
Deleting...
4FGL J1428.3+5635 0.0013592068753496278
Deleting...
4FGL J1428.9+5406 0.001976140800252324
Deleting...
4FGL J1434.5+5428 0.004642516916646855
Deleting...
4FGL J1434.8+6640 -0.00023354684526566416
Deleting...
4FGL J1435.3+7120 -6.925148409209214e-05
Deleting...
4FGL J1435.7+6148 -0.0004448436666280031
Deleting...
4FGL J1436.9+5638 0.002278959953400772
Deleting...
4FGL J1439.7+4958 0.00561972797368071
Deleting...
4FGL J1443.1+5201 0.0013313078052306082
Deleting...
4FGL J1450.8+5201 0.004361339448223589
Deleting...
4FGL J1451.4+6355 -0.002651936993061099
Deleting...
4FGL J1454.0+4927 0.002723105651966762
Deleting...
4FGL J1454.4+5124 0.03152237418180448
Deleting...
4FGL J1454.7+5237 0.0032614193551125936
Deleting...
4FGL J1456.0+5

Since we have achieved convergence, we need to manually add the SwiftJ1644 source to the top of `Swift_ts10.xml` model file.

```xml
<source name="SwiftJ1644" type="PointSource">
 <spectrum type="PowerLaw2">
  <parameter free="true" max="10000.0" min="0.0001" name="Integral" scale="1e-07" value="1.0"/>
  <parameter free="true" max="5.0" min="0.0" name="Index" scale="-1.0" value="2.0"/>
  <parameter free="false" max="500000.0" min="20.0" name="LowerLimit" scale="1.0" value="100.0"/>
  <parameter free="false" max="500000.0" min="20.0" name="UpperLimit" scale="1.0" value="300000.0"/>
 </spectrum>
 <spatialModel type="SkyDirFunction">
  <parameter free="false" max="360.0" min="-360.0" name="RA" scale="1.0" value="251.2054"/>
  <parameter free="false" max="90.0" min="-90.0" name="DEC" scale="1.0" value="57.5808"/>
 </spatialModel>
</source>
```

With the Swift source in the XML file, we can now calculate the upper limit (the paper used an upper energy limit of 10GeV so that's what we are using here).

In [33]:
#help(UnbinnedAnalysis)

In [34]:
like3 = UnbinnedAnalysis(obs,'./data/Swift_ts10_new.xml',optimizer='Minuit')

from UpperLimits import UpperLimits

ul = UpperLimits(like3)
ul['SwiftJ1644'].compute(emin=100,emax=10000)
print(ul['SwiftJ1644'].results)

0 1.0 -13.251780977050657 1.0020512289980601e-07
1 1.4 -8.581677974132617 1.4030658781359018e-07
2 1.7999999999999998 -3.3013048511002125 1.804161226789073e-07
3 2.1999999999999997 2.474914384929434 2.2053187542459382e-07
[2.13e-07 ph/cm^2/s for emin=100.0, emax=10000.0, delta(logLike)=1.35]


Note that this is in ph/cm^2/s and not ergs/cm^2/s.

# Binned Data Upper Limits

In the case of binned data, one needs to follow the standard [likelihood analysis](https://github.com/fermi-lat/AnalysisThreads/blob/master/SourceAnalysis/1.BinnedLikelihood/binned_likelihood_tutorial.ipynb) or the [python version](https://github.com/fermi-lat/AnalysisThreads/blob/master/SourceAnalysis/4.SummedPythonLikelihood/summed_tutorial.ipynb) in order to generate livetime cubes, counts cubes, source maps and exposure maps.

The upper limits steps are nearly identical to the previous section, but one needs to import BinnedAnalysis and use BinnedObs with the proper file format instead:

```python
import pyLikelihood
from BinnedAnalysis import *

obs = BinnedObs(srcMaps='file_name',binnedExpMap='file_name',
expCube ='file_name',irfs='CALDB')
like = BinnedAnalysis(obs,'XML_file_name',optimizer='NewMinuit')
```